In [1]:
import torch
import doubleauction

In [2]:
import torch.nn as nn

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=64, hidden2=64, hidden3=64):
        super(Critic, self).__init__()
        self.bn1 = nn.BatchNorm1d(nb_states)
        self.fc1 = nn.Linear(nb_states, hidden1)
        
        self.bn2 = nn.BatchNorm1d(hidden1+nb_actions)
        self.fc2 = nn.Linear(hidden1+nb_actions, hidden2)
        
        self.bn3 = nn.BatchNorm1d(hidden2)
        self.fc3 = nn.Linear(hidden2, hidden3)
        
        self.bn4 = nn.BatchNorm1d(hidden3)
        self.fc4 = nn.Linear(hidden3, 1)

        self.relu = nn.ReLU()
    
    def forward(self, s, a):
        out = self.fc1(self.bn1(s))
        out = self.relu(out)

        out = self.fc2(self.bn2(torch.cat([out,a],1)))
        out = self.relu(out)
        
        out = self.fc3(self.bn3(out))
        out = self.relu(out)
        
        out = self.fc4(self.bn4(out))
        
        return out
    
    

class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=64, hidden2=64, init_w=3e-3):
        super(Actor, self).__init__()
        self.bn1 = nn.BatchNorm1d(nb_states)
        self.fc1 = nn.Linear(nb_states, hidden1)
        
        self.bn2 = nn.BatchNorm1d(hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        
        self.bn3 = nn.BatchNorm1d(hidden2)
        self.fc3 = nn.Linear(hidden2, nb_actions)
        
        self.relu = nn.ReLU()   
        self.softplus = nn.Softplus()
    
    def forward(self, x):
        out = self.fc1(self.bn1(x))
        out = self.relu(out)
        
        out = self.fc2(self.bn2(out))
        out = self.relu(out)
        out = self.fc3(self.bn3(out))
#         out = self.softplus(out)
        
        return out
    
    

In [3]:
import torch.nn as nn

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=64, hidden2=64, hidden3=64):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(nb_states, hidden1)
        
        self.fc2 = nn.Linear(hidden1+nb_actions, hidden2)
        
        self.fc3 = nn.Linear(hidden2, hidden3)
        self.fc4 = nn.Linear(hidden3, 1)

        self.relu = nn.ReLU()
    
    def forward(self, s, a):
        out = self.fc1(s)
        out = self.relu(out)

        out = self.fc2(torch.cat([out,a],1))
        out = self.relu(out)
        
        out = self.fc3(out)
        out = self.relu(out)
        
        out = self.fc4(out)
    
        return out
    
    

class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=64, hidden2=64, init_w=3e-3):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, hidden1)
        
        self.fc2 = nn.Linear(hidden1, hidden2)
        
        self.fc3 = nn.Linear(hidden2, nb_actions)
        
        self.relu = nn.ReLU()   
        self.softplus = nn.Softplus()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
#         out = self.softplus(out)
        
        return out
    
    

In [4]:

import scipy.stats
import numpy as np
from doubleauction.agents import Buyer, Seller
import torch
        
class DDPGSeller(Seller):
    def __init__(self, agent_id: str,  reservation_price: float, *,
                    max_noise=50., anneal_steps=1e4, min_noise=5.,
                    batch_size = 64, mem_size=100000, lr=1e-2, width_actor=64, width_critic=64,
                    tau=1e-2, discount=0.98, wd = 1e-4):
        """
        A seller agent that extends the market agent
        :param agent_id: a unique id that differentiates this agent to other agents
        :param reservation_price: the reservation price, or minimum price that this agent is
        willing to sell
        """
        super().__init__(agent_id, reservation_price)
        self.max_noise = max_noise
        self.anneal_steps = anneal_steps
        self.min_noise = min_noise
        
        self.sigma = max_noise
        
        self.actor = Actor(5, 1, hidden1=width_actor, hidden2=width_actor)
        self.actor_target = Actor(5, 1, hidden1=width_actor, hidden2=width_actor)
        self.actor_optim  = torch.optim.Adam(self.actor.parameters(), lr=lr, weight_decay=wd)

        self.critic = Critic(5, 1, hidden1=width_critic, hidden2=width_critic)
        self.critic_target = Critic(5, 1, hidden1=width_critic, hidden2=width_critic)
        self.critic_optim  = torch.optim.Adam(self.critic.parameters(), lr=lr, weight_decay=wd)

        self.memory = SequentialMemory(limit=mem_size, window_length=1)
    
        self.tau = tau
        self.lr = lr
        self.width_actor = width_actor
        self.width_critic = width_critic

        self.mem_size = mem_size
        self.batch_size = batch_size
        
        self.discount = discount
        
        self.criterion = nn.MSELoss()
        
        self.eval = False
        
        self.game_count = 0
        
        self.new_game()
        

    
    def new_game(self):
        self.last_demand = 0
        self.game_first = True
        self.round_first = True
        self.last_successful = False
        
        self.state = None
        self.action = None
        
        self.new_round()
        
        if not self.eval:
            self.sigma = self.max_noise - self.game_count * (self.max_noise - self.min_noise) / self.anneal_steps
            self.sigma = max(self.min_noise, self.sigma)
            
        self.game_count += 1
        
    def new_round(self):
        self.round_first = True
        
        
    def decide(self, observations):
        
        ## update state
        self.state = np.array([self.last_demand, self.last_successful,
                               self.game_first, self.round_first,
                               self.reservation_price], dtype=np.float)
        s = torch.tensor(self.state).float().unsqueeze(0)

        with torch.no_grad():
            self.actor.eval()
            if self.eval:
                a = self.actor(s)
                a = a.item()
            else:
                
                n = np.random.normal(0, self.sigma)
                d = self.actor(s).item()
                a = d + n

#                 a = abs(a)
                print(d)
                
            self.actor.train()

        self.action = a
        
        self.last_demand = a
        self.game_first = False
        self.round_first = False

        return self.reservation_price + a
  

    def observe(self, reward, done):
        self.memory.append(self.state, self.action, reward, done)
        self.last_successful = (reward > 0)

        
    def learn(self):
        
        # perform grad descent    
        
        # Sample batch
        state_batch, action_batch, reward_batch, \
        next_state_batch, terminal_batch = self.memory.sample_and_split(self.batch_size)

        # Prepare for the target q batch
        with torch.no_grad():
            next_q_values = self.critic_target(
                torch.tensor(next_state_batch, dtype=torch.float),
                self.actor_target(torch.tensor(next_state_batch, dtype=torch.float)))

            target_q_batch = torch.tensor(reward_batch, dtype=torch.float) + \
                self.discount*torch.tensor(terminal_batch, dtype=torch.float)*next_q_values

        # Critic update
        self.critic.zero_grad()

        q_batch = self.critic(torch.tensor(state_batch, dtype=torch.float), 
                                torch.tensor(action_batch, dtype=torch.float))

        value_loss = self.criterion(q_batch, target_q_batch)
        value_loss.backward()
        self.critic_optim.step()

        # Actor update
        self.actor.zero_grad()

        policy_loss = -self.critic(
            torch.tensor(state_batch, dtype=torch.float),
            self.actor(torch.tensor(state_batch, dtype=torch.float)))

        policy_loss = policy_loss.mean()
        policy_loss.backward()
        self.actor_optim.step()

        # Target update
        soft_update(self.actor_target, self.actor, self.tau)
        soft_update(self.critic_target, self.critic, self.tau)
        

# Create environment

In [5]:
import matplotlib.pyplot as plt

In [6]:
import doubleauction
from doubleauction.agents import RandomSeller, RandomBuyer
from doubleauction.environments import MarketEnvironment
from doubleauction.matchers import RandomMatcher
from doubleauction.util import SequentialMemory, hard_update, soft_update
from doubleauction.util import generate_seller_prices_paper, generate_buyer_prices_paper

In [7]:
records = {}
records['rewards'] = []
records['demands'] = []
records['prices'] = []

In [8]:
rewards = []
epochs = 500
warmup_epochs = 20
seller_agent = DDPGSeller('learner', 0, 
                          discount = 0.97, lr = 3e-4, max_noise=22., min_noise=1.5, anneal_steps=400,
                          wd = 3e-3, mem_size=500000, tau=5e-3)

In [9]:
mdict = torch.load('results/models2')
seller_agent.actor.load_state_dict(mdict['actor'])
seller_agent.actor_target.load_state_dict(mdict['actor'])

seller_agent.critic.load_state_dict(mdict['critic'])
seller_agent.critic_target.load_state_dict(mdict['critic_target'])

<All keys matched successfully>

In [ ]:
%%time
for e in range(epochs):
    seller_agent.reservation_price = generate_seller_prices_paper(1)[0]
    
    sellers = []
    for ii, p in enumerate(generate_seller_prices_paper(19)):
        sellers.append(RandomSeller('s'+str(ii), p))
    sellers.append(seller_agent)

    buyers = []
    for ii, p in enumerate(generate_buyer_prices_paper(20)):
        buyers.append(RandomBuyer('b'+str(ii), p))

    agents = sellers + buyers
    
    seller_agent.new_game()
    
    setting = {
        'self_last_offer': False,
        'same_side_last_offers': False,
        'other_side_last_offers': False,
        'completed_deals': False,
        'current_time': False,
        'max_time': False,
        'n_sellers': False,
        'n_buyers': False
    }

    ROUNDS_PER_GAME = 10
    
    market_env = MarketEnvironment(sellers=sellers, buyers=buyers, max_steps=10,
                                   matcher=RandomMatcher(reward_on_reference=True), setting=setting)
    init_observation = market_env.reset()

    round_avg = 0.
    offer_avg = 0.
    time_avg = 0.
    
    records['demands'].append([])
    records['rewards'].append([])
    records['prices'].append(seller_agent.reservation_price)

    
    for n_round in range(ROUNDS_PER_GAME):
        
        init_observation = market_env.reset()
        observations = {k.agent_id:None for k in agents}
        done = {k.agent_id:False for k in agents}
        reward_hist = []
        rounds = 0
        terminate_round = False
        
        seller_agent.new_round()
        
        records['demands'][-1].append([])
        records['rewards'][-1].append([])
        
        offers_list = []
        
        while not terminate_round:
            offers = {}

            offers = {a.agent_id : a.decide(observations[a.agent_id]) for a in agents}

            observations, rewards, done, _ = market_env.step(offers)
            reward_hist.append(rewards)
            rounds += 1

            terminate_round = all(done.values()) or rounds >= 10 or done['learner']

            # create record of experience
            seller_agent.observe(rewards['learner'], terminate_round and n_round+1==ROUNDS_PER_GAME)
            
            offers_list.append(offers['learner'] - seller_agent.reservation_price)
            
            records['demands'][-1][-1].append(offers['learner'] - seller_agent.reservation_price)
            records['rewards'][-1][-1].append(rewards['learner'])
            
            round_avg += rewards['learner']

            time_avg += 1
    
        offer_avg += sum(offers_list) / len(offers_list)
#         time_vs_rewards.append(round_avg)
#         time_vs_demand.append(sum(offers_list) / len(offers_list))
        
        if e >= warmup_epochs:
            seller_agent.learn()
    
    print('Epoch: {}, Avg. earnings: {}, Avg. demand: {}, Avg. time: {}'.format(e, round_avg / 10., 
                                                                            offer_avg / 10.,
                                                                            time_avg / 10.))
    
    

82.591552734375
15.765169143676758
81.70726013183594
17.772789001464844
83.17229461669922
17.996082305908203
15.114241600036621
66.51066589355469
16.738283157348633
82.98388671875
16.856060028076172
15.070321083068848
82.95304870605469
19.088266372680664
82.888671875
18.80286979675293
63.068275451660156
15.202396392822266
81.90463256835938
18.491960525512695
15.129348754882812
82.24122619628906
15.43398380279541
82.6380615234375
15.505826950073242
82.19448852539062
15.507482528686523
56.41104507446289
16.181001663208008
Epoch: 0, Avg. earnings: 1.082396069175384, Avg. demand: 36.72723631110022, Avg. time: 2.9
93.11326599121094
17.4564266204834
86.10690307617188
18.058652877807617
93.08732604980469
19.914003372192383
93.25019836425781
19.89044189453125
17.08477020263672
93.7042236328125
17.500877380371094
94.423095703125
18.93868064880371
92.6790771484375
19.898406982421875
92.35739135742188
18.526994705200195
92.64033508300781
18.21477699279785
93.84669494628906
18.809045791625977
16.9

109.6343994140625
21.634021759033203
19.93929100036621
110.2421646118164
21.632707595825195
20.031034469604492
110.03799438476562
23.152851104736328
20.172069549560547
110.37225341796875
Epoch: 6, Avg. earnings: -5.843256671403135, Avg. demand: 56.15988547955858, Avg. time: 7.2
88.44422912597656
19.247926712036133
16.138254165649414
89.59249114990234
17.916345596313477
88.45670318603516
17.68122673034668
89.13676452636719
18.060205459594727
16.18906593322754
88.42478942871094
18.88730812072754
88.71400451660156
16.70514488220215
87.78437805175781
17.42101287841797
89.12281036376953
19.67238998413086
89.16325378417969
16.14691734313965
88.80836486816406
19.257387161254883
88.37706756591797
19.77689552307129
89.455322265625
16.262048721313477
89.28329467773438
19.69015121459961
66.12061309814453
16.344757080078125
88.44967651367188
18.177289962768555
88.90402221679688
16.287700653076172
88.73377990722656
18.8359317779541
16.307315826416016
16.157751083374023
89.82740783691406
18.83092117

17.03227424621582
84.19380950927734
19.178438186645508
62.8709602355957
15.701315879821777
15.469199180603027
83.21321105957031
22.151451110839844
15.321009635925293
82.98304748535156
16.213716506958008
84.16161346435547
19.44346809387207
84.21698760986328
18.796892166137695
82.25418090820312
17.70107650756836
15.345126152038574
71.295166015625
16.470762252807617
83.37686920166016
18.325483322143555
Epoch: 15, Avg. earnings: 3.517531313557401, Avg. demand: 50.095337799017315, Avg. time: 4.0
112.703369140625
23.720226287841797
113.53192901611328
23.7293758392334
112.28761291503906
22.26207733154297
96.24382781982422
21.829729080200195
112.82376098632812
20.884174346923828
113.5737533569336
23.2774715423584
113.58224487304688
25.149456024169922
113.83639526367188
22.175790786743164
113.07655334472656
23.463848114013672
114.25198364257812
22.960628509521484
39.32118606567383
20.5687313079834
112.40835571289062
24.57327651977539
20.52034568786621
20.670324325561523
20.445436477661133
20.48

9.277888298034668
39.695457458496094
9.27720832824707
61.654296875
8.640209197998047
60.46754455566406
9.238950729370117
35.346649169921875
55.79161834716797
61.25587463378906
9.172201156616211
62.32621765136719
9.884220123291016
10.360185623168945
9.73465633392334
36.95225143432617
10.71610164642334
62.41559600830078
9.64156436920166
10.637258529663086
36.53533172607422
10.371445655822754
42.73564910888672
25.678268432617188
54.368961334228516
9.94079303741455
48.993473052978516
10.180481910705566
40.365760803222656
41.38148880004883
61.60079574584961
10.966498374938965
55.71059799194336
10.597155570983887
42.20238494873047
59.064964294433594
11.640148162841797
45.56798553466797
12.32119083404541
50.92961883544922
12.10547924041748
42.630340576171875
57.85035705566406
11.795755386352539
52.46405029296875
11.40071964263916
60.143280029296875
60.45966339111328
12.339459419250488
54.052101135253906
34.01485061645508
12.77145767211914
60.34290313720703
56.01740646362305
12.360991477966309

26.046415328979492
14.406027793884277
14.772317886352539
15.449502944946289
18.706554412841797
16.66128158569336
18.567934036254883
18.570087432861328
14.69559383392334
17.99989128112793
Epoch: 39, Avg. earnings: -11.586060230094349, Avg. demand: 11.363857708622623, Avg. time: 4.6
18.85323715209961
17.735185623168945
13.814215660095215
14.461021423339844
19.769073486328125
18.21118927001953
19.283008575439453
15.192620277404785
15.726984977722168
19.269371032714844
21.291704177856445
20.55709457397461
15.63252067565918
17.87473487854004
18.236835479736328
17.972251892089844
14.628613471984863
17.88431739807129
18.30280303955078
17.527713775634766
16.81490707397461
17.624488830566406
17.46068572998047
17.17202377319336
17.569843292236328
15.874537467956543
14.515847206115723
14.427812576293945
15.887792587280273
15.631628036499023
14.012192726135254
14.109498023986816
12.886237144470215
Epoch: 40, Avg. earnings: 0.0980087482096252, Avg. demand: 11.599343250544079, Avg. time: 3.3
17.4011

...

34.51377487182617
15.77898120880127
50.34705352783203
15.409573554992676
39.07674026489258
14.414340019226074
58.03585433959961
18.780231475830078
34.93339920043945
15.259112358093262
39.443885803222656
15.000066757202148
57.27490997314453
19.10943031311035
22.517330169677734
14.743912696838379
54.99984359741211
17.610797882080078
35.220882415771484
14.490260124206543
56.76099395751953
18.514890670776367
44.85230255126953
13.358002662658691
54.45323181152344
16.950021743774414
14.527929306030273
28.33477020263672
13.857900619506836
56.31513214111328
15.793338775634766
54.2432861328125
15.860344886779785
40.63304901123047
13.160696029663086
54.85341262817383
15.868305206298828
32.5530891418457
13.621888160705566
55.85975646972656
14.795140266418457
52.45536804199219
14.802492141723633
55.01356506347656
15.124204635620117
37.06272506713867
13.64093017578125
34.147010803222656
13.413369178771973
55.72705841064453
16.508813858032227
41.1939697265625
12.238563537597656
55.72824478149414
15.

19.62774658203125
7.276254177093506
21.766925811767578
7.472517967224121
21.142955780029297
7.683883190155029
22.5966796875
7.879954814910889
23.59634780883789
8.079058647155762
23.40135955810547
8.26233959197998
23.67295265197754
8.443774223327637
23.993947982788086
8.619872093200684
Epoch: 446, Avg. earnings: 7.189830053240824, Avg. demand: 15.587857467952242, Avg. time: 2.0
25.590909957885742
7.919404029846191
18.5020694732666
9.378752708435059
18.23985481262207
8.356760025024414
17.661182403564453
8.552811622619629
17.30940055847168
8.843029022216797
17.631149291992188
9.935778617858887
17.176258087158203
10.579187393188477
Epoch: 447, Avg. earnings: 11.480790092110885, Avg. demand: 13.779364079399036, Avg. time: 1.4
36.27627944946289
11.379136085510254
58.072265625
9.569283485412598
56.301883697509766
9.547033309936523
58.06980514526367
9.553712844848633
58.10834884643555
9.560518264770508
43.5556640625
10.065643310546875
56.64555358886719
9.787346839904785
57.42330551147461
9.801

34.4968376159668
16.431005477905273
21.619333267211914
16.710851669311523
16.370590209960938
41.77060317993164
14.613408088684082
53.942771911621094
15.734301567077637
31.642560958862305
16.545381546020508
51.30615997314453
14.808149337768555
34.03512191772461
16.53089141845703
40.55463790893555
15.134268760681152
44.916107177734375
15.695548057556152
53.65789794921875
15.070138931274414
35.628910064697266
16.347530364990234
52.78101348876953
14.942218780517578
53.65839767456055
15.085872650146484
53.53151321411133
Epoch: 455, Avg. earnings: 0.0, Avg. demand: 31.759377498666044, Avg. time: 10.0
18.685148239135742
12.110054969787598
36.87662124633789
11.2559814453125
12.90345287322998
13.197787284851074
14.224196434020996
13.568288803100586
14.337385177612305
14.047277450561523
14.090545654296875
14.111602783203125
13.655112266540527
Epoch: 456, Avg. earnings: 13.88799335013087, Avg. demand: 14.726129897755873, Avg. time: 1.3
15.54685115814209
13.179062843322754
12.120705604553223
12.36

51.519142150878906
11.01696491241455
50.979576110839844
11.122840881347656
51.68090057373047
11.049483299255371
38.70793533325195
10.737971305847168
49.59102249145508
10.949179649353027
46.289188385009766
10.837485313415527
51.670631408691406
11.33728313446045
52.126922607421875
11.113855361938477
38.773048400878906
10.61404037475586
51.48761749267578
10.79834270477295
51.80520248413086
10.80213737487793
47.35060501098633
10.72524642944336
51.624996185302734
10.751232147216797
26.4805908203125
12.350384712219238
50.3887825012207
10.70100212097168
52.1117057800293
10.627711296081543
51.56913757324219
10.650157928466797
51.45393371582031
10.633671760559082
37.697914123535156
10.808731079101562
51.92853546142578
10.715923309326172
52.02169418334961
10.591830253601074
52.01604461669922
10.551617622375488
47.29325866699219
10.526862144470215
37.6820182800293
10.916855812072754
51.439876556396484
10.470877647399902
51.68618392944336
10.420475006103516
51.14866638183594
10.452203750610352
Epo

16.541982650756836
16.376367568969727
16.106359481811523
23.972017288208008
18.104446411132812
16.439655303955078
16.347299575805664
16.425138473510742
17.89476203918457
16.61371612548828
16.352914810180664
16.553743362426758
16.55674171447754
16.461530685424805
17.098085403442383
16.623458862304688
16.431241989135742
16.599306106567383
16.327234268188477
16.343713760375977
16.31749153137207
27.200349807739258
17.632694244384766
16.368898391723633
19.111764907836914
16.367197036743164
16.367307662963867
16.630966186523438
16.264171600341797
16.296300888061523
23.242477416992188
16.973886489868164
16.237241744995117
16.272424697875977
17.50275421142578
16.25227928161621
16.756032943725586
16.487470626831055
16.431407928466797
16.676677703857422
16.559003829956055
16.654403686523438
16.272064208984375
16.31285858154297
Epoch: 476, Avg. earnings: 0.0, Avg. demand: 16.90694537216246, Avg. time: 10.0
26.193452835083008
14.872832298278809
15.403911590576172
13.603858947753906


In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

l2 = flatten( records['rewards'])
l3 = [sum(ll) for ll in l2]

plt.figure()
plt.plot(l3)
# plt.plot(smooth(l2, 10))

In [ ]:
torch.save(records, 'results/records3')

torch.save({'actor':seller_agent.actor.state_dict(),
           'actor_target':seller_agent.actor_target.state_dict(),
          'critic':seller_agent.critic.state_dict(),
          'critic_target':seller_agent.critic_target.state_dict()}, 'results/models3')

In [ ]:
torch.save(seller_agent.memory, 'results/memory3')